In [24]:
import urllib, json
import pandas as pd
import re
from itertools import permutations

from pulp import *

In [12]:
preds = pd.read_csv('predictions_2015-09-21.csv')
preds['displayName'] = preds['First Name'] + " " + preds['Last Name']

preds = preds[['Position', 'displayName', 'FPPG', 'Salary']]
preds.columns = ['position', 'displayName', 'points', 'salary']

In [31]:
preds.loc[(preds['position'] == 'C') | (preds['position'] == '1B'), 'position'] = '1B/C'
preds.loc[(preds['position'] == 'CF') | (preds['position'] == 'LF') | (preds['position'] == 'RF'), 'position'] = 'OF'
preds.loc[preds['position'] == 'DH', 'position'] = 'UTIL'

In [35]:
salaries = {}
points = {}
for pos in preds.position.unique():
    available_pos = preds[preds.position == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

pos_num_available = {
    "OF": 3,
    "3B": 1,
    "2B": 1,
    "SS": 1,
    "UTIL": 1,
    "1B/C": 1,
    "P": 1
}

pos_util = {
    "OF": 1,
    "3B": 1,
    "2B": 1,
    "SS": 1,
    "UTIL": 1,
    "1B/C": 1,
    "P": 0
}

pos_util_available = 1

In [33]:
SALARY_CAP = 50000
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [36]:
prob = LpProblem("Fantasy", LpMaximize)
rewards = []
costs = []
position_constraints = []

# Setting up the reward
for k, v in _vars.items():
    costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    prob += lpSum([pos_util[k] * _vars[k][i] for i in v]) <= pos_util_available
    
prob += lpSum(rewards)
prob += lpSum(costs) <= SALARY_CAP

In [37]:
prob.solve()

1

In [53]:
summary(prob)

Variables:

1B_C_joey_votto
2B_brandon_phillips
3B_todd_frazier
OF_yoenis_cespedes
P_jonathon_niese
SS_carlos_correa
UTIL_edwin_encarnacion
---------------------------------------
Predicted Points: 57.55534648895264


In [52]:
def summary(prob):
    div = '---------------------------------------'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name)
    #print(div)
    #print("Constraints:")
    #for constraint in constraints:
        #constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        #if constraint_pretty != "":
            #print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Predicted Points: " + str(eval(score)))